In [1]:
from urllib.request import urlopen

In [2]:
import pandas as pd

In [3]:
import json

In [4]:
URL = "https://services.surfline.com/kbyg/spots/forecasts/{dataset}?spotId={spot_id}&days={nb_days}&intervalHours={interval_hours}"
SPOT_ID = "5842041f4e65fad6a7708c8d"

DATASETS = {"weather", "conditions", "tides", "wind", "wave"}

In [5]:
def get_data(spot_id, dataset, nb_days=6, interval_hours=1):
    assert dataset in DATASETS
    url = URL.format(spot_id=spot_id, dataset=dataset, nb_days=nb_days, interval_hours=interval_hours)
    print(url)
    with urlopen(url) as fdesc:
        data = fdesc.read()
        return json.loads(data.decode("utf8"))

In [6]:
wave_data = get_data(SPOT_ID, "wave")

https://services.surfline.com/kbyg/spots/forecasts/wave?spotId=5842041f4e65fad6a7708c8d&days=6&intervalHours=1


In [7]:
df_associated = pd.io.json.json_normalize(wave_data["associated"])
df_associated.head()

,forecastLocation.lat,forecastLocation.lon,location.lat,location.lon,offshoreLocation.lat,offshoreLocation.lon,units.temperature,units.tideHeight,units.waveHeight,units.windSpeed,utcOffset
0,45.006,-1.237,45.002222,-1.2025,45,-2,C,M,M,KPH,2


In [8]:
df_wave = pd.io.json.json_normalize(wave_data["data"]["wave"])
df_wave.timestamp = pd.to_datetime(df_wave.timestamp, unit='s', utc=True) + pd.Timedelta(hours=2)
df_wave.head()

,surf.max,surf.min,surf.optimalScore,swells,timestamp
0,1.69,1.60,2,"[{'period': 11, 'optimalScore': 0, 'height': 1...",2018-06-16 00:00:00+00:00
1,1.69,1.60,2,"[{'period': 11, 'optimalScore': 0, 'height': 1...",2018-06-16 01:00:00+00:00
2,1.52,1.22,2,"[{'period': 10, 'optimalScore': 0, 'height': 1...",2018-06-16 02:00:00+00:00
3,1.52,1.22,2,"[{'period': 10, 'optimalScore': 0, 'height': 1...",2018-06-16 03:00:00+00:00
4,1.52,1.22,2,"[{'period': 10, 'optimalScore': 0, 'height': 1...",2018-06-16 04:00:00+00:00


In [10]:
for i in range(len(df_wave.swells[0])):
    data_swell_i = pd.io.json.json_normalize(df_wave.swells.apply(lambda x: x[i]))
    data_swell_i.columns = data_swell_i.columns.map(lambda x: "%d_%s" % (i, x))
    df_wave = df_wave.join(data_swell_i, how='left')
df_wave.drop(columns=["swells"], inplace=True)
df_wave.head()

,surf.max,surf.min,surf.optimalScore,timestamp,0_direction,0_height,0_optimalScore,0_period,1_direction,1_height,...,3_optimalScore,3_period,4_direction,4_height,4_optimalScore,4_period,5_direction,5_height,5_optimalScore,5_period
0,1.69,1.60,2,2018-06-16 00:00:00+00:00,299.53,1.62,0,11,327.66,0.27,...,0,0,0,0,0,0,0,0,0,0
1,1.69,1.60,2,2018-06-16 01:00:00+00:00,299.53,1.62,0,11,327.66,0.27,...,0,0,0,0,0,0,0,0,0,0
2,1.52,1.22,2,2018-06-16 02:00:00+00:00,298.13,1.55,0,10,330.47,0.06,...,0,0,0,0,0,0,0,0,0,0
3,1.52,1.22,2,2018-06-16 03:00:00+00:00,298.13,1.55,0,10,330.47,0.06,...,0,0,0,0,0,0,0,0,0,0
4,1.52,1.22,2,2018-06-16 04:00:00+00:00,298.13,1.55,0,10,330.47,0.06,...,0,0,0,0,0,0,0,0,0,0


In [11]:
weather_data = get_data(SPOT_ID, "weather")

https://services.surfline.com/kbyg/spots/forecasts/weather?spotId=5842041f4e65fad6a7708c8d&days=6&intervalHours=1


In [13]:
df_weather = pd.io.json.json_normalize(weather_data["data"]["weather"])
df_weather.timestamp = pd.to_datetime(df_weather.timestamp, unit='s', utc=True) + pd.Timedelta(hours=2)
df_weather.head()

,condition,temperature,timestamp
0,NIGHT_PARTLY_CLOUDY_NO_RAIN,15.9,2018-06-16 00:00:00+00:00
1,NIGHT_MOSTLY_CLOUDY_NO_RAIN,15.7,2018-06-16 01:00:00+00:00
2,NIGHT_MOSTLY_CLOUDY_NO_RAIN,15.4,2018-06-16 02:00:00+00:00
3,NIGHT_MOSTLY_CLOUDY_NO_RAIN,15.5,2018-06-16 03:00:00+00:00
4,NIGHT_MOSTLY_CLOUDY_NO_RAIN,15.4,2018-06-16 04:00:00+00:00


In [14]:
wind_data = get_data(SPOT_ID, "wind")

https://services.surfline.com/kbyg/spots/forecasts/wind?spotId=5842041f4e65fad6a7708c8d&days=6&intervalHours=1


In [16]:
df_wind = pd.io.json.json_normalize(wind_data["data"]["wind"])
df_wind.timestamp = pd.to_datetime(df_wind.timestamp, unit='s', utc=True) + pd.Timedelta(hours=2)
df_wind.head()

,direction,optimalScore,speed,timestamp
0,310.99,0,15.88,2018-06-16 00:00:00+00:00
1,299.61,0,13.75,2018-06-16 01:00:00+00:00
2,284.89,0,12.33,2018-06-16 02:00:00+00:00
3,290.91,0,12.64,2018-06-16 03:00:00+00:00
4,296.59,0,13.08,2018-06-16 04:00:00+00:00


In [18]:
tides_data = get_data(SPOT_ID, "tides")

https://services.surfline.com/kbyg/spots/forecasts/tides?spotId=5842041f4e65fad6a7708c8d&days=6&intervalHours=1


In [20]:
df_tides = pd.io.json.json_normalize(tides_data["data"]["tides"])
df_tides.timestamp = pd.to_datetime(df_tides.timestamp, unit='s', utc=True) + pd.Timedelta(hours=2)
df_tides.head()

,height,timestamp,type
0,1.32,2018-06-16 00:00:00+00:00,NORMAL
1,0.72,2018-06-16 01:00:00+00:00,NORMAL
2,0.67,2018-06-16 01:22:00+00:00,LOW
3,0.81,2018-06-16 02:00:00+00:00,NORMAL
4,1.51,2018-06-16 03:00:00+00:00,NORMAL
